# Traffic Sign Classication using Keras

### Implementation of CNNs in Keras to solve to recognize traffic signs from the German Traffic Sign Dataset.

GitHub: https://github.com/skhiearth/Traffic-Sign-Classification-using-Keras

### Importing the dataset

In [1]:
from urllib.request import urlretrieve
from os.path import isfile
from tqdm import tqdm

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile('train.p'):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='Train Dataset') as pbar:
        urlretrieve(
            'https://s3.amazonaws.com/udacity-sdc/datasets/german_traffic_sign_benchmark/train.p',
            'train.p',
            pbar.hook)

if not isfile('test.p'):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='Test Dataset') as pbar:
        urlretrieve(
            'https://s3.amazonaws.com/udacity-sdc/datasets/german_traffic_sign_benchmark/test.p',
            'test.p',
            pbar.hook)

print('Training and Test data downloaded.')

Training and Test data downloaded.


In [2]:
import pickle
import numpy as np
import math
import tensorflow as tf
from sklearn.utils import shuffle

with open('train.p', 'rb') as f:
    data = pickle.load(f)

X_train = data['features']
y_train = data['labels']

assert (len(X_train)==len(y_train))
print("Loaded training samples.")

X_train, y_train = shuffle(X_train, y_train)

Loaded training samples.


### Preprocessing images

In [3]:
def normalize_grayscale(image_data):
    a = -0.5    
    b = 0.5     
    x_min = 0   
    x_max = 255 
    return (a + ((image_data - x_min) * (b - a) / (x_max - x_min)))

X_normalized = normalize_grayscale(X_train)

# One Hot encode the labels to the variable y_one_hot
from sklearn.preprocessing import LabelBinarizer
y_one_hot = LabelBinarizer().fit_transform(y_train)

### Model definition and training

In [4]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()

# Convolution / Max / Pooling / Dropout
model.add(Convolution2D(32, 5, 5, border_mode='valid', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Activation('relu'))

model.add(Convolution2D(32, 5, 5, border_mode='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Activation('relu'))

# Flatten for a Fully connected layer
model.add(Flatten())

# Fully connected layer 1
model.add(Dense(120))
model.add(Activation('relu'))

# Fully connected layer 2
model.add(Dense(84))
model.add(Activation('relu'))

# Fully connected layer 3
model.add(Dense(43))
model.add(Activation('softmax'))

model.compile('adam', 'categorical_crossentropy', ['accuracy'])

history = model.fit(X_normalized, 
                    y_one_hot, 
                    batch_size=128, 
                    nb_epoch=25, 
                    validation_split=0.2)

Using TensorFlow backend.
/Users/skhiearth/anaconda3/envs/virtual_platform/lib/python3.5/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), input_shape=(32, 32, 3..., padding="valid")`
  if sys.path[0] == '':
/Users/skhiearth/anaconda3/envs/virtual_platform/lib/python3.5/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), padding="valid")`
/Users/skhiearth/anaconda3/envs/virtual_platform/lib/python3.5/site-packages/ipykernel_launcher.py:43: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 31367 samples, validate on 7842 samples
Epoch 1/25
31367/31367 [==============================] - 28s 881us/step - loss: 2.2453 - accuracy: 0.3748 - val_loss: 0.8310 - val_accuracy: 0.7605
Epoch 2/25
31367/31367 [==============================] - 30s 971us/step - loss: 0.7523 - accuracy: 0.7627 - val_loss: 0.3400 - val_accuracy: 0.9202
Epoch 3/25
31367/31367 [==============================] - 29s 911us/step - loss: 0.4751 - accuracy: 0.8529 - val_loss: 0.2819 - val_accuracy: 0.9444
Epoch 4/25
31367/31367 [==============================] - 28s 901us/step - loss: 0.3673 - accuracy: 0.8869 - val_loss: 0.1485 - val_accuracy: 0.9602
Epoch 5/25
31367/31367 [==============================] - 34s 1ms/step - loss: 0.2871 - accuracy: 0.9110 - val_loss: 0.1143 - val_accuracy: 0.9719
Epoch 6/25
31367/31367 [==============================] - 32s 1ms/step - loss: 0.2554 - accuracy: 0.9201 - val_loss: 0.0975 - val_accuracy: 0.9777
Epoch 7/25
31367/31367 [==============================] - 25s

### Saving the model

In [5]:
from keras.models import load_model

model.save('classification_model.h5')
del model

### Testing the model

In [7]:
with open('test.p', 'rb') as f:
    data_test = pickle.load(f)
    
X_test = data_test['features']
y_test = data_test['labels']

# Preprocess data & one-hot encode the labels
X_normalized_test = normalize_grayscale(X_test)
y_one_hot_test = LabelBinarizer().fit_transform(y_test)

# Evaluate model on test data
model = load_model('classification_model.h5')
metrics = model.evaluate(X_normalized_test, y_one_hot_test)

for metric_i in range(len(model.metrics_names)):
    metric_name = model.metrics_names[metric_i]
    metric_value = metrics[metric_i]
    print('{}: {}'.format(metric_name, metric_value))

12630/12630 [==============================] - 3s 225us/step
loss: 0.17921974206887487
accuracy: 0.9553444385528564
